# Part 1

1. Read the case.csv file from the 311 call data into a Spark DataFrame.  
2. How old is the latest (in terms of days past SLA) currently open issue?   
    How long has the oldest (in terms of days since opened) currently opened issue been open?  
3. How many Stray Animal cases are there?  
4. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?  
5. Create a new DataFrame without any information related to dates or location.  
6. Read dept.csv into a Spark DataFrame. Inspect the dept_name column. Replace the missing values with "other".  

# Part 2

1. Convert the council_district column to a string column.  
2. Extract the year from the case_closed_date column.  
3. Convert num_days_late from days to hours in new columns num_hours_late.  
4. Convert the case_late column to a boolean column.  
5. Convert the SLA_days columns to a double column.  
6. Pull it all together

# Part 3

1. Create a DataFrame with all combinations of council_district and service_request_type (regardless of whether the combination is observed in the data).  
2. Join the case data with the source and department data.  
3. Are there any cases that do not have a request source?  

# Part 4

1. Who are the top 10 service request types in terms of number of requests?  
2. Who are the top 10 service request types in terms of average days late?  
3. Does number of days late depend on department?  
4. How do number of days late depend on department division and request type?  

# <span style='color:red'>Part 1</span>

In [45]:
import pyspark

import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import uuid   # Create unique table name
from pyspark.sql.functions import col

import warnings
warnings.filterwarnings("ignore")

from pyspark.sql import SparkSession, DataFrame, Column, Row, GroupedData, \
    DataFrameNaFunctions, DataFrameStatFunctions, functions, types, Window
from pyspark.sql import functions as f

In [2]:
spark = SparkSession.builder.master("local").appName("read").\
    enableHiveSupport().\
    getOrCreate()

In [3]:
df_case = spark.read.csv("sa311/case.csv", sep=",",
                    header=True, inferSchema=True)

In [21]:
df_case.select("case_closed","num_days_late"). \
    filter(df_case.case_closed == "NO").\
    orderBy(df_case.num_days_late.desc(), df_case.case_closed.desc()). \
    show(5)

+-----------+------------------+
|case_closed|     num_days_late|
+-----------+------------------+
|         NO|       348.6458333|
|         NO|       348.6458333|
|         NO|348.52356480000003|
|         NO|347.58256939999995|
|         NO|       345.3894213|
+-----------+------------------+
only showing top 5 rows



In [23]:
df_case.select("service_request_type"). \
    filter(df_case.service_request_type == "Stray Animal").count()

26760

How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [26]:
df_case.select("service_request_type","dept_division"). \
    filter(df_case.dept_division == "Field Operations").\
    filter(df_case.service_request_type != "Officer Standby").count()

113902

Create a new DataFrame without any information related to dates or location.

In [30]:
df_case.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [32]:
df_case.columns

['case_id',
 'case_opened_date',
 'case_closed_date',
 'SLA_due_date',
 'case_late',
 'num_days_late',
 'case_closed',
 'dept_division',
 'service_request_type',
 'SLA_days',
 'case_status',
 'source_id',
 'request_address',
 'council_district']

In [34]:
df_not_dates_address = df_case.select("case_late",
               "num_days_late",
               "case_closed",
               "service_request_type",
               "dept_division",
               "SLA_days", 
               "case_status",
               "source_id",
               "council_district"
              )

Read dept.csv into a Spark DataFrame. Inspect the dept_name column. Replace the missing values with "other".

In [35]:
df_dept = spark.read.csv("sa311/dept.csv", sep=",",
                    header=True, inferSchema=True)

In [38]:
df_dept.show()

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|                null|  DSD/Code Enforcement|                YES|
|   Dangerous Premise|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Dangerous Premise...|Code Enforcement ...|

In [42]:
df_dept.fillna('Other', ["dept_name"]).show()

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|               Other|  DSD/Code Enforcement|                YES|
|   Dangerous Premise|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Dangerous Premise...|Code Enforcement ...|

# <span style='color:red'>Part 2</span>

1. Convert the council_district column to a string column.  
2. Extract the year from the case_closed_date column.  
3. Convert num_days_late from days to hours in new columns num_hours_late.  
4. Convert the case_late column to a boolean column.  
5. Convert the SLA_days columns to a double column.  
6. Pull it all together

In [50]:
df_case.toPandas()

,case_id,case_opened_date,case_closed_date,SLA_due_date,case_late,num_days_late,case_closed,dept_division,service_request_type,SLA_days,case_status,source_id,request_address,council_district
0,1014127332,1/1/18 0:42,1/1/18 12:29,9/26/20 0:42,NO,-998.508762,YES,Field Operations,Stray Animal,999.000000,Closed,svcCRMLS,"2315 EL PASO ST, San Antonio, 78207",5
1,1014127333,1/1/18 0:46,1/3/18 8:11,1/5/18 8:30,NO,-2.012604,YES,Storm Water,Removal Of Obstruction,4.322222,Closed,svcCRMSS,"2215 GOLIAD RD, San Antonio, 78223",3
2,1014127334,1/1/18 0:48,1/2/18 7:57,1/5/18 8:30,NO,-3.022338,YES,Storm Water,Removal Of Obstruction,4.320729,Closed,svcCRMSS,"102 PALFREY ST W, San Antonio, 78223",3
3,1014127335,1/1/18 1:29,1/2/18 8:13,1/17/18 8:30,NO,-15.011481,YES,Code Enforcement,Front Or Side Yard Parking,16.291887,Closed,svcCRMSS,"114 LA GARDE ST, San Antonio, 78223",3
4,1014127336,1/1/18 1:34,1/1/18 13:29,1/1/18 4:34,YES,0.372164,YES,Field Operations,Animal Cruelty(Critical),0.125000,Closed,svcCRMSS,"734 CLEARVIEW DR, San Antonio, 78228",7
5,1014127337,1/1/18 6:28,1/1/18 14:38,1/31/18 8:30,NO,-29.743981,YES,Signals,Traffic Signal Ops and Maintenance,30.084468,Closed,svcCRMSS,BANDERA RD and BRESNAHAN,7
6,1014127338,1/1/18 6:57,1/2/18 15:32,1/17/18 8:30,NO,-14.706736,YES,Code Enforcement,Front Or Side Yard Parking,16.064294,Closed,svcCRMSS,"10133 FIGARO CANYON, San Antonio, 78251",4
7,1014127339,1/1/18 6:58,1/2/18 15:32,1/17/18 8:30,NO,-14.706620,YES,Code Enforcement,Front Or Side Yard Parking,16.063796,Closed,svcCRMSS,"10133 FIGARO CANYON, San Antonio, 78251",4
8,1014127340,1/1/18 6:58,1/2/18 15:32,1/17/18 8:30,NO,-14.706620,YES,Code Enforcement,Right Of Way/Sidewalk Obstruction,16.063333,Closed,svcCRMSS,"10133 FIGARO CANYON, San Antonio, 78251",4
9,1014127341,1/1/18 6:59,1/2/18 15:32,1/17/18 8:30,NO,-14.706493,YES,Code Enforcement,Front Or Side Yard Parking,16.062859,Closed,svcCRMSS,"10133 FIGARO CANYON, San Antonio, 78251",4


In [44]:
df_case.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [48]:
df_case \
    .withColumn("council_district", df_case["council_district"].getItem(0).cast("string"))

AnalysisException: "Can't extract value from council_district#23: need struct type but got int;"